In [ ]:
pip install emoji


In [ ]:
pip install num2words

In [ ]:
pip install pyspellchecker

In [ ]:
import re
import string
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

import warnings
warnings.filterwarnings('ignore')


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.model_selection import train_test_split, ParameterGrid

# Bag-of words
from sklearn.feature_extraction.text import CountVectorizer

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# BOW
from sklearn.feature_extraction.text import CountVectorizer

# Classifiers
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score,f1_score,confusion_matrix, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, GridSearchCV

import string, re, nltk
import emoji
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from num2words import num2words
from spellchecker import SpellChecker
from nltk.stem.porter import PorterStemmer
import spacy
from nltk.stem import WordNetLemmatizer


In [ ]:
# train data
train = pd.read_csv("/content/train.csv")
train.head()

In [ ]:
train.shape

In [ ]:
train['toxic'].value_counts()

In [ ]:
#test the data without labels
test = pd.read_csv('/content/test.csv')
test.head()

In [ ]:
#test data labels
test_labels = pd.read_csv('/content/test_labels.csv')
test_labels.head()

In [ ]:
test_labels['toxic'].value_counts()

The target features consist of three values: -1, 0, and 1. To analyze this further, we need to investigate the test data labels. Since this is a binary classification problem, the labels should ideally be 0 or 1. We need to unde

In [ ]:
test_labels[test_labels['toxic']==1]

In [ ]:
test[test['id']== '0013fed3aeae76b7']['comment_text']


In [ ]:
test_labels[test_labels['toxic']==-1]

In [ ]:
test[test['id']== '00001cee341fdb12']['comment_text']

In [ ]:
test[test['id']== 'ffffce3fb183ee80']['comment_text']

After analyzing the data, we observed that labels with values of -1 and 1 appear to represent the same category. This suggests that -1 might be a data entry error or an incorrect label.

# ***EDA***

In [ ]:
train['comment_len'] = train['comment_text'].apply(lambda x: len(x.split(' ')))
train.head()

In [ ]:
train[train['comment_len'].isnull()]

In [ ]:
toxic_df = train[train['toxic']==1].copy()
severe_toxic_df = train[train['severe_toxic']==1].copy()
obscene_toxic_df = train[train['obscene']==1].copy()
insult_df = train[train['insult']==1].copy()
identity_hate_df = train[train['identity_hate']==1].copy()
non_toxic_df = train[(train['toxic']==0) &
                  (train['severe_toxic']==0) &
                  (train['obscene']==0) &
                  (train['insult']==0) &
                 (train['identity_hate']==0)].copy()

In [ ]:
# Set Seaborn style
sns.set(style='whitegrid')

# Create a figure with 2 rows and 2 columns of subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 10))
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

# Define unique colors for each plot
colors = ['red','#1fd655']

# List of dataframes, their corresponding columns, titles, and colors
data_frames = [
    (toxic_df, 'toxic', 'Toxic', colors[0]),
    (severe_toxic_df, 'severe_toxic', 'Severe Toxic', colors[0]),
    (obscene_toxic_df, 'obscene', 'Obscene', colors[0]),
    (insult_df, 'insult', 'Insult', colors[0]),
    (identity_hate_df, 'identity_hate', 'Identity Hate', colors[0]),
    (non_toxic_df, 'toxic', 'non toxic', colors[1]),
]

# Plot each dataframe in a separate subplot
for ax, (df, col, title, color) in zip(axes, data_frames):
    sns.boxplot(x=col, y='comment_len', data=df, ax=ax, color=color)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('Comment Length')
    ax.grid(True, linestyle='--', alpha=0.7)

# Hide the last subplot if using a 2x3 grid
for j in range(len(data_frames), len(axes)):
    axes[j].axis('off')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
categories = [
    'Toxic',
    'Severe Toxic',
    'Obscene',
    'Insult',
    'Identity Hate',
    'Non Toxic'
]

# Corresponding lengths of each dataframe
lengths = [
    len(toxic_df),
    len(severe_toxic_df),
    len(obscene_toxic_df),
    len(insult_df),
    len(identity_hate_df),
    len(non_toxic_df)
]

# Create a DataFrame for plotting
data = pd.DataFrame({
    'Category': categories,
    'Count': lengths
})

# Set Seaborn style
sns.set(style='whitegrid')

# Define colors
colors = ['#ff4c4c'] * 5  # Red color for all categories except Non Toxic
colors.append('#DDF2D1')  # Green color for Non Toxic

# Create the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Category', y='Count', data=data, palette=colors)

# Add titles and labels
ax.set_title('Count of Comments by Category', fontsize=16, fontweight='bold')
ax.set_xlabel('Category', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Load the mask image
comment_mask = np.array(Image.open('/content/comment.png'))

# Function to generate word cloud
def generate_wordcloud(text, title, ax, colormap):
    wc = WordCloud(
        background_color='white',
        max_words=200,
        mask=comment_mask,
        colormap=colormap,
        contour_color='black',
        contour_width=1
    )
    wc.generate(text)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(title, fontdict={'size': 22, 'verticalalignment': 'bottom'})
    ax.axis('off')

# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(24, 15))
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

# Define the categories and their colormaps
categories = ['toxic', 'severe_toxic', 'obscene', 'insult', 'identity_hate', 'non_toxic']
dataframes = [toxic_df, severe_toxic_df, obscene_toxic_df, insult_df, identity_hate_df, non_toxic_df]
colormaps = {
    'toxic': 'plasma',  # Purple to red
    'severe_toxic': 'plasma',
    'obscene': 'plasma',
    'insult': 'plasma',
    'identity_hate': 'plasma',
    'non_toxic': 'summer'  # Green to yellow
}

# Generate word clouds for each category and add to subplots
for i, (category, df) in enumerate(zip(categories, dataframes)):
    text = ' '.join(df['comment_text'])
    colormap = colormaps.get(category, 'viridis')  # Default to 'viridis' if category not found
    generate_wordcloud(text, f'Top words for {category.replace("_", " ").capitalize()}', axes[i], colormap)

# Adjust layout and show plot
plt.tight_layout()
plt.show()

# ***Data Preparartion***

In [ ]:
df = train.copy()

In [ ]:
df['target'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0 ).astype(int) #.astype(int): Converts the Boolean result (True or False) to integers (1 or 0).
df = df[['comment_text', 'target']]
df.head()

In [ ]:
# Calculating the percentage of each class
percentage = df['target'].value_counts(normalize=True) * 100

# Define colors for the bars
colors = ['#DDF2D1' if x == 0 else 'red' for x in percentage.index]

# Plotting the percentage of each class
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=percentage.index, y=percentage, palette=colors)
plt.title('Percentage of Toxic and Non Toxic')
plt.xlabel('Toxic')
plt.ylabel('Percentage (%)')
plt.xticks(ticks=[0, 1], labels=['Non Toxic', 'Toxic'])
plt.yticks(ticks=range(0, 80, 10))

# Displaying the percentage on the bars
for i, p in enumerate(percentage):
    ax.text(i, p + 0.5, f'{p:.2f}%', ha='center', va='bottom')

plt.show()

==> the data is imbalanced

# ***Train-Test Split***

In [ ]:
# Feature-target split
X = df['comment_text']
y = df['target']

# Train-test split (from complete data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data_train = pd.concat([X_train, y_train], axis=1)

# Validation-test split (from test data)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=121)
data_val, data_test = pd.concat([X_val, y_val], axis=1), pd.concat([X_test, y_test], axis=1)

# Comparison of sizes of training set, validation set, and test set
values = np.array([len(data_train), len(data_val), len(data_test)])
labels = ['Training Set', 'Validation Set', 'Test Set']

# Define shades of red
colors = ['#ff3333','#ff9999', '#ff6666']

# Create the pie chart
fig, ax = plt.subplots(figsize=(10, 6))
ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, pctdistance=0.85, colors=colors)

# Draw a white circle at the center to create a donut chart
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig.gca().add_artist(centre_circle)

# Equal aspect ratio ensures that pie is drawn as a circle
ax.axis('equal')

# Add a title
plt.title('Comparison of Sizes of Training Set, Validation Set, and Test Set', y=1.05)

# Show the plot
plt.show()

In [ ]:
# RegexpTokenizer
regexp = RegexpTokenizer("[\w']+")

In [ ]:
# Converting to lowercase
def convert_to_lowercase(text):
    return text.lower()

text = "This is a FUNCTION that CoNvErTs a Text to lowercase"
print("Input: {}".format(text))
print("Output: {}".format(convert_to_lowercase(text)))

In [ ]:
# Removing whitespaces
def remove_whitespace(text):
    return text.strip()

text = " \t This is a string \t "
print("Input: {}".format(text))
print("Output: {}".format(remove_whitespace(text)))

In [ ]:
# Removing punctuations
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "") # discarding apostrophe from the string to keep the contractions intact
    return text.translate(str.maketrans("", "", punct_str))

text = "Here's [an] example? {of} &a string. with.? punctuations!!!!"
print("Input: {}".format(text))
print("Output: {}".format(remove_punctuation(text)))

In [ ]:
# Removing HTML tags
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

text = ' <a href = "https://www.kaggle.com/code/vishalnaik/intensity-classification"> Intensity classification </a>'
print("Input: {}".format(text))
print("Output: {}".format(remove_html(text)))

In [ ]:
"""
😊 - Smiling Face with Smiling Eyes
😂 - Face with Tears of Joy
😍 - Heart Eyes
😒 - Unamused Face
😢 - Crying Face
😎 - Smiling Face with Sunglasses
🙁 - Slightly Frowning Face
👍 - Thumbs Up
👏 - Clapping Hands
🙏 - Folded Hands
❤️ - Red Heart
🥺 - Pleading Face
🤔 - Thinking Face
🥳 - Partying Face
😔 - Pensive Face
😠 - Angry Face
🤗 - Hugging Face
😅 - Smiling Face with Sweat
😡 - Pouting Face
🤩 - Star-Struck
🙌 - Raising Hands
🔥 - Fire
💔 - Broken Heart
🎉 - Party Popper
💪 - Flexed Biceps

"""

# Define a mapping from emoji names to simplified descriptions
emoji_mapping = {
    'smiling_face_with_smiling_eyes': 'mocking',
    'face_with_tears_of_joy': 'sarcastic laugh',
    'heart_eyes': 'obsession',
    'crying_face': 'whining',
    'angry_face': 'rage',
    'enraged_face': 'fury',
    'thumbs_up': 'sarcastic approval',
    'clapping_hands': 'mocking applause',
    'red_heart': 'fake sympathy',
    'face_with_rolling_eyes': 'disdain',
    'face_with_symbols_on_mouth': 'cursing',
    'middle_finger': 'fuck off',
    # Add more mappings as needed
}

# Function to convert emojis to simplified text descriptions
def convert_emoji_to_text(text):
    # Convert emojis to detailed descriptions
    detailed_text = emoji.demojize(text, delimiters=(":", ":"))

    # Replace detailed descriptions with simplified ones
    for detailed, simplified in emoji_mapping.items():
        detailed_text = detailed_text.replace(f':{detailed}:', simplified + ' ')

    return detailed_text

text = "This innovative hd printing technique results in durable and spectacular looking prints 😊😂😡🖕❤️"
print("Input: {}".format(text))
print("Output: {}".format(convert_emoji_to_text(text)))

In [ ]:
# Removing other unicode characters
def remove_http(text):
    http = "https?://\S+|www\.\S+" # matching strings beginning with http (but not just "http")
    pattern = r"({})".format(http) # creating pattern
    return re.sub(pattern, "", text)

text = "It's a function that removes links starting with http: or https such as https://en.wikipedia.org/wiki/Unicode_symbols"
print("Input: {}".format(text))
print("Output: {}".format(remove_http(text)))

In [ ]:
acronyms_dict = {
    "aka": "also known as",
    "asap": "as soon as possible",
    "brb": "be right back",
    "btw": "by the way",
    "dob": "date of birth",
    "faq": "frequently asked questions",
    "fyi": "for your information",
    "idk": "i don't know",
    "idc": "i don't care",
    "iirc": "if i recall correctly",
    "imo": "in my opinion",
    "irl": "in real life",
    "lmk": "let me know",
    "lol": "laugh out loud",
    "ngl": "not gonna lie",
    "noyb": "none of your business",
    "nvm": "never mind",
    "ofc": "of course",
    "omg": "oh my god",
    "pfa": "please find attached",
    "rofl": "rolling on the floor laughing",
    "stfu": "shut the fuck up",
    "tba": "to be announced",
    "tbc": "to be continued",
    "tbd": "to be determined",
    "tbh": "to be honest",
    "ttyl": "talk to you later",
    "wtf": "what the fuck",
    "wth": "what the heck"
}
print("Example: Original form of the acronym 'fyi' is '{}'".format(acronyms_dict['fyi']))

In [ ]:
# List of acronyms
acronyms_list = list(acronyms_dict.keys())

# Function to convert contractions in a text
def convert_acronyms(text):
    words = []
    for word in regexp.tokenize(text):
        if word in acronyms_list:
            words = words + acronyms_dict[word].split()
        else:
            words = words + word.split()

    text_converted = " ".join(words)
    return text_converted

text = "btw you've to fill in the details including dob"
print("Input: {}".format(text))
print("Output: {}".format(convert_acronyms(text)))

In [ ]:
contractions_dict = {
    "'aight": "alright",
    "ain't": "are not",
    "amn't": "am not",
    "arencha": "are not you",
    "aren't": "are not",
    "'bout": "about",
    "can't": "cannot",
    "cap'n": "captain",
    "'cause": "because",
    "'cept": "except",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "dammit": "damn it",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "doesn't": "does not",
    "doin'": "doing",
    "don't": "do not",
    "dunno": "do not know",
    "d'ye": "do you",
    "e'en": "even",
    "e'er": "ever",
    "'em": "them",
    "everybody's": "everybody is",
    "everyone's": "everyone is",
    "fo'c'sle": "forecastle",
    "finna": "fixing to",
    "'gainst": "against",
    "g'day": "good day",
    "gimme": "give me",
    "giv'n": "given",
    "gonna": "going to",
    "gon't": "go not",
    "gotcha": "got you",
    "gotta": "got to",
    "gtg": "got to go",
    "hadn't": "had not",
    "had've": "had have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'll": "he shall",
    "helluva": "hell of a",
    "he's": "he is",
    "here's": "here is",
    "he've": "he have",
    "how'd": "how would",
    "howdy": "how do you do",
    "how'll": "how will",
    "how're": "how are",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i shall",
    "i'm": "i am",
    "imma": "i am about to",
    "i'm'a": "i am about to",
    "i'm'o": "i am going to",
    "innit": "is it not",
    "ion": "i do not",
    "i've": "i have",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i shall",
    "i'm": "i am",
    "i'm'a": "i am about to",
    "i'm'o": "i am going to",
    "innit": "is it not",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'll": "it shall",
    "it's": "it is",
    "iunno": "i do not know",
    "kinda": "kind of",
    "let's": "let us",
    "li'l": "little",
    "ma'am": "madam",
    "mayn't": "may not",
    "may've": "may have",
    "methinks": "me thinks",
    "mightn't": "might not",
    "might've": "might have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "must've": "must have",
    "'neath": "beneath",
    "needn't": "need not",
    "nal": "and all",
    "ne'er": "never",
    "o'clock": "of the clock",
    "o'er": "over",
    "ol'": "old",
    "oughtn't": "ought not",
    "'round": "around",
    "'s": "is",
    "shalln't": "shall not",
    "shan't": "shall not",
    "she'd": "she had",
    "she'll": "she shall",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "somebody's": "somebody is",
    "someone's": "someone is",
    "something's": "something is",
    "so're": "so are",
    "so's": "so is",
    "so've": "so have",
    "that'll": "that shall",
    "that're": "that are",
    "that's": "that is",
    "that'd": "that would",
    "there'd": "there had",
    "there'll": "there shall",
    "there're": "there are",
    "there's": "there is",
    "these're": "these are",
    "these've": "these have",
    "they'd": "they had",
    "they'll": "they shall",
    "they're": "they are",
    "they've": "they have",
    "this's": "this is",
    "those're": "those are",
    "those've": "those have",
    "'thout": "without",
    "'til": "until",
    "'tis": "it is",
    "to've": "to have",
    "'twas": "it was",
    "'tween": "between",
    "'twhere": "it were",
    "wanna": "want to",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we shall",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "whatcha": "what are you",
    "what'd": "what did",
    "what'll": "what shall",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "where'd": "where did",
    "where'll": "where shall",
    "where're": "where are",
    "where's": "where is",
    "where've": "where have",
    "which'd": "which had",
    "which'll": "which shall",
    "which're": "which are",
    "which's": "which is",
    "which've": "which have",
    "who'd": "who would",
    "who'd've": "who would have",
    "who'll": "who shall",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "why'd": "why did",
    "why're": "why are",
    "why's": "why is",
    "willn't": "will not",
    "won't": "will not",
    "wonnot": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd've": "you all would have",
    "y'all'd'n't've": "you all would not have",
    "y'all're": "you all are",
    "y'all'ren't": "you all are not",
    "y'at": "you at",
    "yes'm": "yes madam",
    "yessir": "yes sir",
    "you'd": "you had",
    "you'll": "you shall",
    "you're": "you are",
    "you've": "you have",
    "aight": "alright",
    "aint": "are not",
    "amnt": "am not",
    "arent": "are not",
    "cant": "cannot",
    "cause": "because",
    "couldve": "could have",
    "couldnt": "could not",
    "couldntve": "could not have",
    "darent": "dare not",
    "daresnt": "dare not",
    "dasnt": "dare not",
    "didnt": "did not",
    "doesnt": "does not",
    "doin": "doing",
    "dont": "do not",
    "eer": "ever",
    "everybodys": "everybody is",
    "everyones": "everyone is",
    "finna": "fixing to",
    "gday": "good day",
    "givn": "given",
    "gont": "go not",
    "hadnt": "had not",
    "hadve": "had have",
    "hasnt": "has not",
    "havent": "have not",
    "hed": "he had",
    "hell": "he shall",
    "hes": "he is",
    "heve": "he have",
    "howd": "how did",
    "howdy": "how do you do",
    "howll": "how will",
    "howre": "how are",
    "hows": "how is",
    "idve": "i would have",
    "ill": "i shall",
    "im": "i am",
    "ima": "i am about to",
    "imo": "i am going to",
    "innit": "is it not",
    "ive": "i have",
    "isnt": "is not",
    "itd": "it would",
    "itll": "it shall",
    "its": "it is",
    "lets": "let us",
    "lil": "little",
    "maam": "madam",
    "maynt": "may not",
    "mayve": "may have",
    "methinks": "me thinks",
    "mightnt": "might not",
    "mightve": "might have",
    "mustnt": "must not",
    "mustntve": "must not have",
    "mustve": "must have",
    "neednt": "need not",
    "neer": "never",
    "oclock": "of the clock",
    "oer": "over",
    "ol": "old",
    "oughtnt": "ought not",
    "shallnt": "shall not",
    "shant": "shall not",
    "shed": "she had",
    "shell": "she shall",
    "shes": "she is",
    "shouldve": "should have",
    "shouldnt": "should not",
    "shouldntve": "should not have",
    "somebodys": "somebody is",
    "someones": "someone is",
    "somethings": "something is",
    "thatll": "that shall",
    "thatre": "that are",
    "thatd": "that would",
    "thered": "there had",
    "therell": "there shall",
    "therere": "there are",
    "theres": "there is",
    "thesere": "these are",
    "theseve": "these have",
    "theyd": "they had",
    "theyll": "they shall",
    "theyre": "they are",
    "theyve": "they have",
    "thiss": "this is",
    "thosere": "those are",
    "thoseve": "those have",
    "tis": "it is",
    "tove": "to have",
    "twas": "it was",
    "wanna": "want to",
    "wasnt": "was not",
    "wed": "we had",
    "wedve": "we would have",
    "were": "we are",
    "weve": "we have",
    "werent": "were not",
    "whatd": "what did",
    "whatll": "what shall",
    "whatre": "what are",
    "whats": "what is",
    "whatve": "what have",
    "whens": "when is",
    "whered": "where did",
    "wherell": "where shall",
    "wherere": "where are",
    "wheres": "where is",
    "whereve": "where have",
    "whichd": "which had",
    "whichll": "which shall",
    "whichre": "which are",
    "whichs": "which is",
    "whichve": "which have",
    "whod": "who would",
    "whodve": "who would have",
    "wholl": "who shall",
    "whore": "who are",
    "whos": "who is",
    "whove": "who have",
    "whyd": "why did",
    "whyre": "why are",
    "whys": "why is",
    "wont": "will not",
    "wouldve": "would have",
    "wouldnt": "would not",
    "wouldntve": "would not have",
    "yall": "you all",
    "yalldve": "you all would have",
    "yallre": "you all are",
    "youd": "you had",
    "youll": "you shall",
    "youre": "you are",
    "youve": "you have",
    "'re": "are",
    "that's": "that is",
    "thats": "that is"
}

In [ ]:
# List of contractions
contractions_list = list(contractions_dict.keys())

# Function to convert contractions in a text
def convert_contractions(text):
    words = []
    for word in regexp.tokenize(text):
        if word in contractions_list:
            words = words + contractions_dict[word].split()
        else:
            words = words + word.split()

    text_converted = " ".join(words)
    return text_converted

text = "it's function that make can't removes i'm"

print("Input: {}".format(text))
print("Output: {}".format(convert_contractions(text)))

In [ ]:
abbreviations = {
    "$" : "dollar",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired",
    "lol" : "laugh out loud",
    "lit": "exciting",
    "btw":"by the way",
}

In [ ]:
nltk.download('punkt')

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [abbreviations[word.lower()] if word.lower() in abbreviations else word for word in tokens]
    text = ' '.join(tokens)
    return text

text = "w/o lol, that party was so lit last night! BTW, did you see how many people were taking selfies?"
print("Input: {}".format(text))
print("Output: {}".format(convert_abbrev_in_text(text)))

In [ ]:
# pyspellchecker
spell = SpellChecker()

def pyspellchecker(text):
    word_list = regexp.tokenize(text)
    word_list_corrected = []
    for word in word_list:
        if word in spell.unknown(word_list):
            word_corrected = spell.correction(word)
            if word_corrected == None:
                word_list_corrected.append(word)
            else:
                word_list_corrected.append(word_corrected)
        else:
            word_list_corrected.append(word)
    text_corrected = " ".join(word_list_corrected)
    return text_corrected

text = "I'm goinng therre"
print("Input: {}".format(text))
print("Output: {}".format(pyspellchecker(text)))

In [ ]:
# Stemming
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem

text = "Introducing lemmatization as an improvement over stemming"
print("Input: {}".format(text))
print("Output: {}".format(text_stemmer(text)))

In [ ]:
# Lemmatization

spacy_lemmatizer = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])
#lemmatizer = WordNetLemmatizer()

def text_lemmatizer(text):
    text_spacy = " ".join([token.lemma_ for token in spacy_lemmatizer(text)])
    #text_wordnet = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text)]) # regexp.tokenize(text)
    return text_spacy
    #return text_wordnet

text = "Introducing lemmatization as an improvement over stemming"
print("Input: {}".format(text))
print("Output: {}".format(text_lemmatizer(text)))

In [ ]:
# Discardment of non-alphabetic words
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

text = "It is an ocean of thousands and 1000s of crowd"
print("Input: {}".format(text))
print("Output: {}".format(discard_non_alpha(text)))

In [ ]:
# Ensure the necessary NLTK data is downloaded
nltk.download('averaged_perceptron_tagger')

# Create a RegexpTokenizer instance
regexp = RegexpTokenizer(r'\w+')

def keep_pos(text):
    tokens = regexp.tokenize(text)
    tokens_tagged = nltk.pos_tag(tokens)
    #keep_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'FW']
    keep_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRPS','JJ',
                 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP','WP$',
                 'VBZ', 'WDT', 'WP', 'WPS', 'WRB']

    keep_words = [x[0] for x in tokens_tagged if x[1] in keep_tags]
    return " ".join(keep_words)

text = "I’M sorry If I Change But You Changed Too"
print("Input: {}".format(text))
tokens = regexp.tokenize(text)
print("Tokens: {}".format(tokens))
tokens_tagged = nltk.pos_tag(tokens)
print("Tagged Tokens: {}".format(tokens_tagged))
print("Output: {}".format(keep_pos(text)))

In [ ]:
nltk.download('stopwords')
stops = stopwords.words("english") # stopwords
alphabets = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
all_stops =  stops + alphabets


print(all_stops)

In [ ]:
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in all_stops])

text = "i am sad. i got fail in one subject, wait i will share you on whatsapp"
print("Input: {}".format(text))
print("Output: {}".format(remove_stopwords(text)))

In [ ]:
#Work Flow Integration
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text) # converting text to one line
    text = re.sub(r'\.com\b', '', text) # Remove ".com"
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = convert_emoji_to_text(text)
    text = convert_acronyms(text)
    text = convert_contractions(text)
    text = convert_abbrev_in_text(text)
#     text = pyspellchecker(text)
    text = text_lemmatizer(text) # text = text_stemmer(text)
    text = discard_non_alpha(text)
    text = keep_pos(text)
    text = convert_to_lowercase(text)
    text = remove_stopwords(text)

    return text

text = """'I’M sorry 😡🖕❤️ If I Change But You Changed Too. & We'll [#* combine all ] {functions <into 1> SINGLE 1000 FUNCTION  & apply  on @product #content make
 removes  https://en.wikipedia.org/wiki/Text_normalization"""


print("Input: {}".format(text))
print("Output: {}".format(text_normalizer(text)))

In [ ]:
# Implementing text normalization
data_train_norm, data_val_norm, data_test_norm = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

data_train_norm['normalized_comment_text'] = data_train['comment_text'].apply(text_normalizer)
data_val_norm['normalized_comment_text'] = data_val['comment_text'].apply(text_normalizer)
data_test_norm['normalized_comment_text'] = data_test['comment_text'].apply(text_normalizer)

data_train_norm['target'] = data_train['target']
data_val_norm['target'] = data_val['target']
data_test_norm['target'] = data_test['target']

data_train_norm['normalized_comment_text']

In [ ]:
from wordcloud import WordCloud

# Function to generate word cloud
def generate_wordcloud(text, title, ax):
    wc = WordCloud(
        background_color='white',
        max_words=200,
        colormap='plasma',
        contour_color='black',
        contour_width=1
    )
    wc.generate(text)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(title, fontdict={'size': 22, 'verticalalignment': 'bottom'})
    ax.axis('off')

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(24, 20))

# Define the categories and corresponding datasets
categories = ['Training', 'Validation', 'Test']
datasets = [data_train_norm, data_val_norm, data_test_norm]

# Generate word clouds for each category and add to subplots
for i, (category, dataset) in enumerate(zip(categories, datasets)):
    text = " ".join(description for description in dataset['normalized_comment_text'])
    generate_wordcloud(text, f'Normalized words for {category.capitalize()}', axes[i])

# Adjust layout and show plot
plt.tight_layout()
plt.show()

*** TF-IDF Vectorization***

In [ ]:
# Features and labels
X_train_norm, y_train = data_train_norm['normalized_comment_text'].tolist(), data_train_norm['target'].tolist()
X_val_norm, y_val = data_val_norm['normalized_comment_text'].tolist(), data_val_norm['target'].tolist()
X_test_norm, y_test = data_test_norm['normalized_comment_text'].tolist(), data_test_norm['target'].tolist()

In [ ]:
# TF-IDF vectorization
# TfidfVec = TfidfVectorizer()
TfidfVec = TfidfVectorizer(ngram_range = (1, 2),max_features=400) # using both unigram & bi-grams

X_train_tfidf = TfidfVec.fit_transform(X_train_norm)
X_val_tfidf = TfidfVec.transform(X_val_norm)
X_test_tfidf = TfidfVec.transform(X_test_norm)

***Model Building***

In [ ]:
# Classifiers
names = [
    "ComplementNB",
    "Logistic Regression",
    "KNN Classifier",
    "Decision Tree",
    "Linear SVM",
    "Random Forest",
    "SGD Classifier",
    "Ridge Classifier",
]

models = [
    ComplementNB(),
    LogisticRegression(max_iter = 1000,class_weight= 'balanced'),
    KNeighborsClassifier(n_neighbors = 149, n_jobs = -1),
    DecisionTreeClassifier(class_weight= 'balanced'),
    svm.SVC(kernel = 'linear', class_weight= 'balanced'),
    RandomForestClassifier(n_estimators = 100, class_weight= 'balanced'),
    SGDClassifier(loss = 'hinge',class_weight= 'balanced'),
    RidgeClassifier(class_weight= 'balanced'),
]

In [ ]:
# Function to return summary of baseline models
def score(X_train, y_train, X_test, y_test, X_val, y_val, names=names, models=models):
    score_df = pd.DataFrame()
    score_train, score_test, score_val = [], [], []

    for model in models:
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        y_val_pred = model.predict(X_val)

        score_train.append(f1_score(y_train, y_train_pred, average='weighted'))
        score_test.append(f1_score(y_test, y_test_pred, average='weighted'))
        score_val.append(f1_score(y_val, y_val_pred, average='weighted'))

    score_df["Classifier"] = names
    score_df["Training F1-score"] = score_train
    score_df["Test F1-score"] = score_test
    score_df["Validation F1-score"] = score_val

    score_df.sort_values(by='Validation F1-score', ascending=False, inplace=True)
    return score_df

In [ ]:
# Summary of baseline models
score(X_train_tfidf, y_train,X_test_tfidf, y_test, X_val_tfidf, y_val, names = names, models = models)

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, X_val=None, y_val=None):
    # Evaluate the model on the train data
    print("\n\t  Classification report for training set")
    print("-"*55)
    print(classification_report(y_train, model.predict(X_train), target_names=['Non-toxic', 'toxic']))

    # Evaluate the model on the test data
    print("\n\t   Classification report for test set")
    print("-"*55)
    print(classification_report(y_test, model.predict(X_test), target_names=['Non-toxic', 'toxic']))

    if X_val is not None and y_val is not None:
        # Evaluate the model on the validation data
        print("\n\t   Classification report for validation set")
        print("-"*55)
        print(classification_report(y_val, model.predict(X_val), target_names=['Non-toxic', 'toxic']))

    # Generate confusion matrix for test data
    cm = confusion_matrix(y_test, model.predict(X_test))

    # Define the class labels
    class_labels = ['Non-toxic', 'toxic']

    # Plot the confusion matrix as a heatmap
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix - Test Set')

    # Plot ROC curve for each class on the test data
    plt.subplot(1, 2, 2)
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(len(class_labels)):
        fpr[i], tpr[i], _ = roc_curve(y_test, model.predict_proba(X_test)[:, i], pos_label=i)
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], lw=2, label=f'{class_labels[i]} (AUC = {roc_auc[i]:0.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Test Set')
    plt.legend(loc='lower right')
    plt.tight_layout()

    plt.show()

    if X_val is not None and y_val is not None:
        # Generate confusion matrix for validation data
        cm_val = confusion_matrix(y_val, model.predict(X_val))

        # Plot the confusion matrix as a heatmap
        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', cbar=False,
                    xticklabels=class_labels, yticklabels=class_labels)
        plt.title('Confusion Matrix - Validation Set')

        # Plot ROC curve for each class on the validation data
        plt.subplot(1, 2, 2)
        fpr_val = {}
        tpr_val = {}
        roc_auc_val = {}
        for i in range(len(class_labels)):
            fpr_val[i], tpr_val[i], _ = roc_curve(y_val, model.predict_proba(X_val)[:, i], pos_label=i)
            roc_auc_val[i] = auc(fpr_val[i], tpr_val[i])
            plt.plot(fpr_val[i], tpr_val[i], lw=2, label=f'{class_labels[i]} (AUC = {roc_auc_val[i]:0.2f})')
        plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve - Validation Set')
        plt.legend(loc='lower right')
        plt.tight_layout()

        plt.show()

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, class_weight= 'balanced')
rf_model = rf.fit(X_train_tfidf, y_train)

In [ ]:
evaluate_model(rf_model, X_train_tfidf, y_train, X_test_tfidf, y_test, X_val_tfidf, y_val)

In [ ]:
test

In [ ]:
test_labels

In [ ]:
filtered_test_labels = test_labels[(test_labels != -1).all(axis=1)]

In [ ]:
test_df = pd.merge(test, filtered_test_labels, on='id', how='inner')
test_df['target'] = (test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0 ).astype(int)
test_df = test_df[['comment_text', 'target']]
test_df.head()

In [ ]:
test_df['target'].value_counts()

In [ ]:
# Implementing text normalization
data_new_norm = pd.DataFrame()

data_new_norm['normalized_comment_text'] = test_df['comment_text'].apply(text_normalizer)
data_new_norm['target'] = test_df['target']
X_new_norm, y_new = data_new_norm['normalized_comment_text'].tolist(), data_new_norm['target'].tolist()

In [ ]:
# TF-IDF vectorization
X_new_tfidf = TfidfVec.transform(X_new_norm)

In [ ]:
y_new_predict = rf_model.predict(X_new_tfidf)


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_new, y_new_predict)

print(f"New Data Accuracy using RF Model: {accuracy:.4f}")